# Crossvalidation experiments on KuHar Resampled to 20Hz

This notebook will perform crossvalidation experiments using the KuHar dataset at 20 Hz as training dataset. It will contain the following steps:

1. Quick load train, test and validation CSV subsets from the balanced KuHar dataset at 20 Hz using `PandasDatasetsIO` helper
2. Quick load train, test and validation CSV subsets from other relevant datasets using `PandasDatasetsIO` helper
3. Subclassing the `Dataset` interface using `PandasMultiModalDataset`
4. Apply the fourier transform on all the datasets
5. Train SVM, KNN and Random Forest classification models on the KuHar dataset in both time and frequency domains
6. Evaluate SVM, KNN and Random Forest classification models on other dataset in both time and frequency domains

The experiments will evaluate the performance of SVM, KNN and RF models trained on a balanced KuHar dataset and tested on other datasets in both time and frequency domains.

## Common imports and definitions

In [1]:
from pathlib import Path  # For defining dataset Paths
import sys                # For include librep package

# This must be done if librep is not installed via pip,
# as this directory (examples) is appart from librep package root
sys.path.append("..")

# Third party imports
import pandas as pd
import numpy as np

# Librep imports
from librep.utils.dataset import PandasDatasetsIO          # For quick load train, test and validation CSVs
from librep.datasets.multimodal import PandasMultiModalDataset # Wrap CSVs to librep's `Dataset` interface

2022-09-13 13:16:46.875383: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-13 13:16:46.875405: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading data from base dataset
Change the path here to use in other dataset as base

In [2]:
# Path for KuHar resampled to 30Hz view with the same activities (and labels numbers)
# It is assumed that the directory will contain (train.csv, test.csv and validation.csv)
dataset_path = Path("../data/views/KuHar/resampled_view_20Hz")

Once paths is defined, we can load the CSV as pandas dataframes

In [3]:
# Kuhar dataframes
train_base, validation_base, test_base = PandasDatasetsIO(dataset_path).load()

Letś take a look in the train dataframes

In [4]:
train_base.head()

,Unnamed: 0,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,...,gyro-z-59,accel-start-time,gyro-start-time,accel-end-time,gyro-end-time,activity code,length,serial,index,user
0,0,0.001911,-0.014536,0.005845,0.003675,-0.014972,0.025607,0.000478,-0.031141,-0.014827,...,0.004456,23.235,23.223,26.260,26.249,0,300,1,2100,1051
1,1,0.004114,-0.003186,0.000759,0.012450,-0.032074,0.007270,-0.000470,0.006980,0.021400,...,0.002979,56.292,56.292,59.245,59.245,0,300,1,5700,1037
2,2,-0.011282,-0.002432,-0.003199,0.008152,-0.021763,0.000309,-0.004968,-0.009551,0.001497,...,0.003343,27.268,27.267,30.290,30.291,0,300,1,2700,1075
3,3,-0.009241,-0.004666,0.021606,-0.007200,0.003091,0.001630,0.005057,-0.008149,0.013167,...,-0.002053,39.421,39.420,42.441,42.440,0,300,6,3900,1008
4,4,-0.013083,-0.005612,0.001645,0.006823,-0.004159,0.000415,0.008178,0.002637,-0.000827,...,0.002603,23.703,23.703,26.656,26.656,0,300,1,2400,1038


## Creating a Librep dataset from pandas dataframes

Change the features to use in other datasets

In [5]:
# Kuhar features to select
features = [
    "accel-x",
    "accel-y",
    "accel-z",
    "gyro-x",
    "gyro-y",
    "gyro-z"
]

combined_df = pd.concat([train_base, validation_base, test_base], ignore_index=True)
df2 = combined_df.query("`activity code` == 0 or `activity code` == 1 or `activity code` == 11 or `activity code` == 14  or `activity code` == 15 or `activity code` == 16")
df2.loc[ df2['activity code'] == 0, 'activity code'] = 3
df2.loc[ df2['activity code'] == 1, 'activity code'] = 2
df2.loc[ df2['activity code'] == 11, 'activity code'] = 4
df2.loc[ df2['activity code'] == 14, 'activity code'] = 5
df2.loc[ df2['activity code'] == 15, 'activity code'] = 1
df2.loc[ df2['activity code'] == 16, 'activity code'] = 0


In [6]:
df2.head()

,Unnamed: 0,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,...,gyro-z-59,accel-start-time,gyro-start-time,accel-end-time,gyro-end-time,activity code,length,serial,index,user
0,0,0.001911,-0.014536,0.005845,0.003675,-0.014972,0.025607,0.000478,-0.031141,-0.014827,...,0.004456,23.235,23.223,26.260,26.249,3,300,1,2100,1051
1,1,0.004114,-0.003186,0.000759,0.012450,-0.032074,0.007270,-0.000470,0.006980,0.021400,...,0.002979,56.292,56.292,59.245,59.245,3,300,1,5700,1037
2,2,-0.011282,-0.002432,-0.003199,0.008152,-0.021763,0.000309,-0.004968,-0.009551,0.001497,...,0.003343,27.268,27.267,30.290,30.291,3,300,1,2700,1075
3,3,-0.009241,-0.004666,0.021606,-0.007200,0.003091,0.001630,0.005057,-0.008149,0.013167,...,-0.002053,39.421,39.420,42.441,42.440,3,300,6,3900,1008
4,4,-0.013083,-0.005612,0.001645,0.006823,-0.004159,0.000415,0.008178,0.002637,-0.000827,...,0.002603,23.703,23.703,26.656,26.656,3,300,1,2400,1038


In [7]:
# Creating the train dataset

combined_train_dset = PandasMultiModalDataset(
    df2,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

## Inspect sample

In [8]:
# Lets print the first sample of combined_train dataset.
# Is a tuple, with an vector of 540 elements as first element and the label as second
x = combined_train_dset[0]
print(x)

(array([ 1.91093286e-03, -1.45361925e-02,  5.84452385e-03,  3.67495627e-03,
       -1.49718059e-02,  2.56068907e-02,  4.77538088e-04, -3.11405362e-02,
       -1.48270261e-02,  7.69834863e-03,  1.06101665e-02, -5.96475630e-02,
       -3.35511310e-03, -1.65885925e-03,  3.94389738e-02, -4.28711994e-02,
       -4.65577088e-03, -1.44686791e-02, -7.36948774e-03, -3.87024460e-03,
        6.24744252e-02, -1.79626835e-02,  3.22744928e-03, -3.75961022e-03,
        1.46163449e-02, -1.07502353e-02, -9.27218103e-03,  5.06417325e-03,
        1.40691624e-02,  1.60138354e-02, -5.34838152e-02, -3.29858611e-03,
        2.31031426e-02,  2.27906805e-02,  2.54595798e-03,  1.75255266e-02,
       -5.10498318e-03, -2.07463519e-02,  1.32902011e-02,  1.37572046e-02,
        7.17675958e-03, -2.01445217e-02,  5.47817384e-03, -7.66570074e-04,
        1.94831071e-02, -1.11694213e-03,  2.27235363e-02, -1.49616813e-02,
       -9.71672954e-03, -7.12839038e-03,  9.02811373e-03, -1.57676951e-03,
       -5.51378813e-03, 

In [9]:
# Inspecting sample
print(f"The sample 0: {x[0]}")
print(f"Shape of sample 0: {x[0].shape}")
print(f"The label of sample 0: {x[1]}")

The sample 0: [ 1.91093286e-03 -1.45361925e-02  5.84452385e-03  3.67495627e-03
 -1.49718059e-02  2.56068907e-02  4.77538088e-04 -3.11405362e-02
 -1.48270261e-02  7.69834863e-03  1.06101665e-02 -5.96475630e-02
 -3.35511310e-03 -1.65885925e-03  3.94389738e-02 -4.28711994e-02
 -4.65577088e-03 -1.44686791e-02 -7.36948774e-03 -3.87024460e-03
  6.24744252e-02 -1.79626835e-02  3.22744928e-03 -3.75961022e-03
  1.46163449e-02 -1.07502353e-02 -9.27218103e-03  5.06417325e-03
  1.40691624e-02  1.60138354e-02 -5.34838152e-02 -3.29858611e-03
  2.31031426e-02  2.27906805e-02  2.54595798e-03  1.75255266e-02
 -5.10498318e-03 -2.07463519e-02  1.32902011e-02  1.37572046e-02
  7.17675958e-03 -2.01445217e-02  5.47817384e-03 -7.66570074e-04
  1.94831071e-02 -1.11694213e-03  2.27235363e-02 -1.49616813e-02
 -9.71672954e-03 -7.12839038e-03  9.02811373e-03 -1.57676951e-03
 -5.51378813e-03 -3.61854449e-03 -9.16178207e-03  1.69776410e-02
 -3.65341848e-03 -2.29361283e-02 -2.51271512e-03  3.50588067e-02
  1.3409791

## Fourier Transform

In [10]:
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT

In [11]:
fft_transform = FFT(centered = True)
transformer = TransformMultiModalDataset(transforms=[fft_transform], new_window_name_prefix="fft.")

### Use FFT in the base dataset

In [12]:
combined_train_dset_fft = transformer(combined_train_dset)

In [13]:
combined_train_dset[:][0]

array([[ 1.91093286e-03, -1.45361925e-02,  5.84452385e-03, ...,
         3.45654902e-03,  2.32869360e-03,  4.45589801e-03],
       [ 4.11395657e-03, -3.18646610e-03,  7.58931558e-04, ...,
        -9.94428406e-04, -1.82853273e-03,  2.97903419e-03],
       [-1.12820040e-02, -2.43180090e-03, -3.19908050e-03, ...,
         3.56838998e-03,  4.38234273e-03,  3.34301636e-03],
       ...,
       [-2.98870193e+00, -1.39523386e-01, -6.52441382e+00, ...,
        -7.48947780e-02, -3.92595416e-01, -2.17587752e-01],
       [ 7.96914982e+00,  1.20087272e+01, -3.34976629e+00, ...,
         1.08870693e-01,  4.74598944e-01,  2.26891463e-02],
       [-8.22046480e+00, -4.99540410e+00,  1.68743679e+01, ...,
         6.03348395e-01,  2.79978155e-01, -1.70569178e-01]])

In [14]:
combined_train_dset_fft[:][0]

array([[2.08923330e-02, 1.12081089e-01, 6.03699767e-02, ...,
        6.86907330e-03, 1.25349286e-02, 1.69158661e-02],
       [1.53802877e-02, 8.24343989e-02, 4.18766153e-02, ...,
        3.72912157e-03, 3.98584265e-03, 1.71193131e-02],
       [5.21272671e-02, 4.82816195e-02, 8.93573044e-02, ...,
        9.75422945e-03, 2.66463902e-02, 7.84359780e-03],
       ...,
       [1.66125362e+01, 6.98974508e+00, 1.66400551e+01, ...,
        7.46678423e-01, 9.79780208e-01, 1.13686391e+00],
       [2.83744371e+01, 2.64833653e+01, 2.09319955e+01, ...,
        1.59587407e+00, 2.04678099e+00, 1.38148423e+00],
       [2.12598609e+01, 1.75026493e+01, 1.66059199e+01, ...,
        1.06153533e+00, 3.67620249e+00, 4.62240026e+00]])

## Loading data from a target dataset
In this case we use the MotionSense dataset resampled at 30 Hz

In [15]:
dataset_path = Path("../data/views/MotionSense/resampled_view_20Hz")

In [16]:
# Kuhar dataframes
train_target, validation_target, test_target = PandasDatasetsIO(dataset_path).load()

In [17]:
train_base.head()

,Unnamed: 0,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,...,gyro-z-59,accel-start-time,gyro-start-time,accel-end-time,gyro-end-time,activity code,length,serial,index,user
0,0,0.001911,-0.014536,0.005845,0.003675,-0.014972,0.025607,0.000478,-0.031141,-0.014827,...,0.004456,23.235,23.223,26.260,26.249,0,300,1,2100,1051
1,1,0.004114,-0.003186,0.000759,0.012450,-0.032074,0.007270,-0.000470,0.006980,0.021400,...,0.002979,56.292,56.292,59.245,59.245,0,300,1,5700,1037
2,2,-0.011282,-0.002432,-0.003199,0.008152,-0.021763,0.000309,-0.004968,-0.009551,0.001497,...,0.003343,27.268,27.267,30.290,30.291,0,300,1,2700,1075
3,3,-0.009241,-0.004666,0.021606,-0.007200,0.003091,0.001630,0.005057,-0.008149,0.013167,...,-0.002053,39.421,39.420,42.441,42.440,0,300,6,3900,1008
4,4,-0.013083,-0.005612,0.001645,0.006823,-0.004159,0.000415,0.008178,0.002637,-0.000827,...,0.002603,23.703,23.703,26.656,26.656,0,300,1,2400,1038


In [18]:
# MotionSense features to select
features = [
    "userAcceleration.x",
    "userAcceleration.y",
    "userAcceleration.z",
    "rotationRate.x",
    "rotationRate.y",
    "rotationRate.z"
]

# Creating the train dataset

combined_target_dset = PandasMultiModalDataset(
    pd.concat([train_target, validation_target, test_target], ignore_index=True),
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

In [19]:
# Lets print the first sample of combined_target dataset.
# Is a tuple, with an vector of 180 elements as first element and the label as second
x = combined_target_dset[0]
print(x)

(array([-1.01580660e-01, -2.21355124e-01, -2.34015566e-01, -2.64551900e-01,
       -2.00990656e-01, -2.08961907e-01, -3.08408466e-01,  8.99426214e-02,
       -3.82515554e-01, -7.75533852e-02,  1.46318283e-01, -4.93770905e-02,
       -2.09640201e-02, -3.17307189e-02, -3.74259464e-01, -1.52182310e-01,
        1.05233312e-01,  1.11319758e-01, -5.38330158e-02, -3.14136807e-01,
       -2.08472528e-01, -3.23492282e-01, -1.91815150e-01, -1.88533126e-01,
       -4.71212234e-01, -3.96779870e-01,  5.07970584e-01,  5.27352561e-02,
        1.59006497e-01,  3.15998485e-02, -1.98139909e-01,  2.08691866e-01,
       -1.08432318e-01, -4.06142257e-01,  1.66143634e-01,  7.70059198e-06,
       -5.77953517e-02, -5.52501402e-02, -2.20866812e-01, -3.14509600e-01,
        2.84967949e-01,  1.34890424e-01,  6.60941177e-02,  6.91335728e-02,
       -3.98362725e-01,  6.46615090e-02,  1.24072879e-01, -2.26239499e-01,
       -3.26437858e-01, -1.39575067e-01, -1.09824066e-02, -7.93132478e-02,
       -4.57364775e-01, 

In [20]:
# Inspecting sample
print(f"The sample 0: {x[0]}")
print(f"Shape of sample 0: {x[0].shape}")
print(f"The label of sample 0: {x[1]}")

The sample 0: [-1.01580660e-01 -2.21355124e-01 -2.34015566e-01 -2.64551900e-01
 -2.00990656e-01 -2.08961907e-01 -3.08408466e-01  8.99426214e-02
 -3.82515554e-01 -7.75533852e-02  1.46318283e-01 -4.93770905e-02
 -2.09640201e-02 -3.17307189e-02 -3.74259464e-01 -1.52182310e-01
  1.05233312e-01  1.11319758e-01 -5.38330158e-02 -3.14136807e-01
 -2.08472528e-01 -3.23492282e-01 -1.91815150e-01 -1.88533126e-01
 -4.71212234e-01 -3.96779870e-01  5.07970584e-01  5.27352561e-02
  1.59006497e-01  3.15998485e-02 -1.98139909e-01  2.08691866e-01
 -1.08432318e-01 -4.06142257e-01  1.66143634e-01  7.70059198e-06
 -5.77953517e-02 -5.52501402e-02 -2.20866812e-01 -3.14509600e-01
  2.84967949e-01  1.34890424e-01  6.60941177e-02  6.91335728e-02
 -3.98362725e-01  6.46615090e-02  1.24072879e-01 -2.26239499e-01
 -3.26437858e-01 -1.39575067e-01 -1.09824066e-02 -7.93132478e-02
 -4.57364775e-01 -1.14570859e-01  2.62746376e-01 -1.53196322e-01
  3.57891496e-01 -2.31041538e-01 -2.26239680e-01  3.22787316e-01
 -1.4633490

In [21]:
combined_target_dset_fft = transformer(combined_target_dset)

In [22]:
combined_target_dset[:][0]

array([[-0.10158066, -0.22135512, -0.23401557, ...,  0.29260805,
         0.932535  ,  0.78214699],
       [-0.08252731,  0.20113614, -0.017408  , ...,  0.71842028,
         0.33650671,  0.93656304],
       [ 0.10832292, -0.04594076,  0.01741017, ...,  0.66747733,
        -0.0253858 , -0.03709332],
       ...,
       [-0.09988536, -0.06214647,  0.00353448, ..., -0.02418191,
        -0.26655296, -0.81136345],
       [ 0.28012977,  0.39306956, -0.22951424, ...,  0.2010255 ,
        -0.37453148,  0.13791192],
       [ 0.18094204,  0.19997285,  0.24716494, ..., -0.1108572 ,
         0.10921315,  0.77653174]])

In [23]:
combined_target_dset_fft[:][0]

array([[5.2349672 , 1.31500631, 0.62294979, ..., 1.51949579, 1.40946664,
        2.67359214],
       [0.4195556 , 1.47781888, 0.7555246 , ..., 3.16244675, 1.15978979,
        1.87787114],
       [2.1285244 , 1.03841388, 0.40157264, ..., 0.31071895, 2.31689217,
        1.23138236],
       ...,
       [0.8358572 , 0.86086449, 0.62594271, ..., 4.97735758, 0.22642244,
        1.06843498],
       [2.0848148 , 1.94485535, 1.3993648 , ..., 1.84977362, 3.0305433 ,
        1.57749012],
       [1.0167636 , 0.96328948, 0.90865235, ..., 2.80883343, 2.55697575,
        1.59127442]])

## Train and evaluate Random Forest classifier

In [24]:
from librep.utils.workflow import SimpleTrainEvalWorkflow, MultiRunWorkflow
from librep.estimators import RandomForestClassifier
from librep.metrics.report import ClassificationReport
import yaml

reporter = ClassificationReport(use_accuracy=True, use_f1_score=True, use_classification_report=False, use_confusion_matrix=False, plot_confusion_matrix=False)
experiment = SimpleTrainEvalWorkflow(estimator=RandomForestClassifier, estimator_creation_kwags ={'n_estimators':100} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

In [25]:
result = multi_run_experiment(combined_train_dset, combined_target_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663075012.762969
    result:
    -   accuracy: 0.3638888888888889
        f1 score (macro): 0.2459885681613974
        f1 score (micro): 0.3638888888888888
        f1 score (weighted): 0.4817892096163804
    run id: 1
    start: 1663075011.2075016
    time taken: 1.5554673671722412
-   end: 1663075013.8672519
    result:
    -   accuracy: 0.36626984126984125
        f1 score (macro): 0.2520585900810498
        f1 score (micro): 0.36626984126984125
        f1 score (weighted): 0.48048109245863285
    run id: 2
    start: 1663075012.762971
    time taken: 1.104280948638916
-   end: 1663075014.9599817
    result:
    -   accuracy: 0.3611111111111111
        f1 score (macro): 0.24616923533153232
        f1 score (micro): 0.3611111111111111
        f1 score (weighted): 0.47605298689068987
    run id: 3
    start: 1663075013.8672538
    time taken: 1.0927278995513916



In [26]:
result = multi_run_experiment(combined_train_dset_fft, combined_target_dset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663075015.651434
    result:
    -   accuracy: 0.4527777777777778
        f1 score (macro): 0.39288707007762774
        f1 score (micro): 0.4527777777777778
        f1 score (weighted): 0.5126684854779279
    run id: 1
    start: 1663075014.9650984
    time taken: 0.686335563659668
-   end: 1663075016.3417087
    result:
    -   accuracy: 0.4442460317460317
        f1 score (macro): 0.39060647035324364
        f1 score (micro): 0.4442460317460317
        f1 score (weighted): 0.4978855931388199
    run id: 2
    start: 1663075015.6514359
    time taken: 0.6902728080749512
-   end: 1663075017.0167215
    result:
    -   accuracy: 0.4452380952380952
        f1 score (macro): 0.390419826655754
        f1 score (micro): 0.4452380952380952
        f1 score (weighted): 0.5000563638204364
    run id: 3
    start: 1663075016.3417103
    time taken: 0.675011157989502



## Train and evaluate Support Vector Machine classifier

In [27]:
#from librep.estimators import SVC
from sklearn.svm import SVC

experiment = SimpleTrainEvalWorkflow(estimator=SVC, estimator_creation_kwags ={'C':3.0, 'kernel':"rbf"} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

result = multi_run_experiment(combined_train_dset, combined_target_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663075018.6871042
    result:
    -   accuracy: 0.2398809523809524
        f1 score (macro): 0.1239765562545554
        f1 score (micro): 0.2398809523809524
        f1 score (weighted): 0.3557853485073493
    run id: 1
    start: 1663075017.0223165
    time taken: 1.664787769317627
-   end: 1663075020.22852
    result:
    -   accuracy: 0.2398809523809524
        f1 score (macro): 0.1239765562545554
        f1 score (micro): 0.2398809523809524
        f1 score (weighted): 0.3557853485073493
    run id: 2
    start: 1663075018.687106
    time taken: 1.5414140224456787
-   end: 1663075022.0779395
    result:
    -   accuracy: 0.2398809523809524
        f1 score (macro): 0.1239765562545554
        f1 score (micro): 0.2398809523809524
        f1 score (weighted): 0.3557853485073493
    run id: 3
    start: 1663075020.2285216
    time taken: 1.8494179248809814



In [28]:
result = multi_run_experiment(combined_train_dset_fft, combined_target_dset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663075022.7352297
    result:
    -   accuracy: 0.1986111111111111
        f1 score (macro): 0.13600524016714766
        f1 score (micro): 0.1986111111111111
        f1 score (weighted): 0.2612169820550746
    run id: 1
    start: 1663075022.0830193
    time taken: 0.6522104740142822
-   end: 1663075023.376182
    result:
    -   accuracy: 0.1986111111111111
        f1 score (macro): 0.13600524016714766
        f1 score (micro): 0.1986111111111111
        f1 score (weighted): 0.2612169820550746
    run id: 2
    start: 1663075022.7352314
    time taken: 0.6409506797790527
-   end: 1663075024.0167615
    result:
    -   accuracy: 0.1986111111111111
        f1 score (macro): 0.13600524016714766
        f1 score (micro): 0.1986111111111111
        f1 score (weighted): 0.2612169820550746
    run id: 3
    start: 1663075023.3761837
    time taken: 0.6405777931213379



## Train and evaluate K Neighbors Classifier classifier

In [29]:
#from librep.estimators import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

experiment = SimpleTrainEvalWorkflow(estimator=KNeighborsClassifier, estimator_creation_kwags ={'n_neighbors' :1} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

result = multi_run_experiment(combined_train_dset, combined_target_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663075024.3832588
    result:
    -   accuracy: 0.18551587301587302
        f1 score (macro): 0.09614999354572819
        f1 score (micro): 0.18551587301587302
        f1 score (weighted): 0.27488175248601787
    run id: 1
    start: 1663075024.0225675
    time taken: 0.3606913089752197
-   end: 1663075024.5891175
    result:
    -   accuracy: 0.18551587301587302
        f1 score (macro): 0.09614999354572819
        f1 score (micro): 0.18551587301587302
        f1 score (weighted): 0.27488175248601787
    run id: 2
    start: 1663075024.3832612
    time taken: 0.2058563232421875
-   end: 1663075024.7972252
    result:
    -   accuracy: 0.18551587301587302
        f1 score (macro): 0.09614999354572819
        f1 score (micro): 0.18551587301587302
        f1 score (weighted): 0.27488175248601787
    run id: 3
    start: 1663075024.589119
    time taken: 0.20810627937316895



In [30]:
result = multi_run_experiment(combined_train_dset_fft, combined_target_dset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663075024.9549575
    result:
    -   accuracy: 0.21388888888888888
        f1 score (macro): 0.1495498387542211
        f1 score (micro): 0.21388888888888888
        f1 score (weighted): 0.2782279390235567
    run id: 1
    start: 1663075024.8031933
    time taken: 0.1517641544342041
-   end: 1663075025.0953264
    result:
    -   accuracy: 0.21388888888888888
        f1 score (macro): 0.1495498387542211
        f1 score (micro): 0.21388888888888888
        f1 score (weighted): 0.2782279390235567
    run id: 2
    start: 1663075024.9549596
    time taken: 0.140366792678833
-   end: 1663075025.2662492
    result:
    -   accuracy: 0.21388888888888888
        f1 score (macro): 0.1495498387542211
        f1 score (micro): 0.21388888888888888
        f1 score (weighted): 0.2782279390235567
    run id: 3
    start: 1663075025.095328
    time taken: 0.17092108726501465

